In [1]:
## Make relevant imports
import numpy as np
import pandas as pd
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

In [2]:
## Import relevant python files
from evalMain import groupNDCG

In [3]:
## Train the individual rec.sys.
# get the data
data = pd.read_csv('../dataset/clean/user_ratings_neg_1000_20_20_1_train.csv', delimiter=',')
data = data.drop(columns=['Unnamed: 0'])

# construct dataframe in format (user, item, rating) via column addition
df_ui = data.rename(columns={"UserID": "user", "JobID": "item", "Rating":"rating"})
# check data being read properly
display(df_ui.head(10)) 

# train UserUser collaborative filterring
user_user = UserUser(10, min_nbrs=3)  # Minimum (3) and maximum (10) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_ui)

,user,item,rating
0,554,640492,1
1,554,280275,1
2,554,747584,1
3,554,25820,1
4,554,600058,1
5,554,196603,1
6,554,1113088,1
7,554,40,1
8,554,627377,1
9,554,957,1


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Numba is using threading layer omp - consider TBB
found 1 potential runtime problems - see https://boi.st/lkpy-perf


In [6]:
## Read dataframe generated by group aggregation (groupAggregation.ipynb)
df_grpAggr = pd.read_csv('../groupAgg/grpAggr.csv', delimiter=',')
df_grpAggr = df_grpAggr.drop(columns=['Unnamed: 0'])

In [7]:
## Compute ndcg for a group
grp_results = pd.DataFrame(columns=["mean","min","max"])

for i, row in df_grpAggr.iterrows():
    g_mean, g_min, g_max = groupNDCG(row["Members"], row["Recommendation"], recsys) # error inside
    grp_results.loc[len(grp_results.index)] = [g_mean, g_min, g_max] 

display(grp_results)
display(df_grpAggr)



--- Logging error ---
Traceback (most recent call last):
  File "/Users/kirillgugunishvili/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3790, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 152, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 160, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index_class_helper.pxi", line 70, in pandas._libs.index.Int64Engine._check_type
KeyError: '['

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/kirillgugunishvili/opt/anaconda3/lib/python3.9/site-packages/lenskit/algorithms/user_knn.py", line 256, in _get_user_data
    upos = self.user_index_.get_loc(user)
  File "/Users/kirillgugunishvili/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3797, in get_loc
    raise KeyError(key) from err
KeyError: '['

During handling of the above exception, another exc

,mean,min,max
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


,Members,Recommendation
0,"[1165352, 668454, 838185, 1056891, 578645, 950...","[33567, 1030251, 1001363, 1062379, 475364]"
1,"[549231, 583914, 941975, 294558, 803135, 99662...","[467462, 758674, 764913, 175827, 805775]"
2,"[410122, 277505, 692599, 783536, 1420225, 5545...","[403015, 322071, 759309, 283718, 1815]"
3,"[192178, 850445, 439070, 1340921, 473967, 1327...","[1047062, 241110, 189081, 229937, 194305]"
4,"[988920, 485759, 481324, 1294945, 669787, 1060...","[282017, 510956, 768776, 886344, 870248]"
